In [71]:
import xgboost as xgb
import numpy as np
import os 
from pathlib import Path
import pickle 
import pandas as pd
from sklearn.metrics import roc_auc_score
import datetime
WORK_PATH = str(Path( os.path.abspath('')).parent.parent)
feat_dict = pickle.load(open(WORK_PATH + "/results/features_category.pkl" ,"rb"))

numeric_cols = feat_dict['numeric_cols']
cat_cols = feat_dict['cat_cols']

print(cat_cols)
print(numeric_cols)

clf = xgb.XGBClassifier(
                max_depth = 10,
                n_estimators=1000, # should not too large otherwise overfitting
                learning_rate=0.035, # not too small, not too large
                nthread=4,
                subsample=0.8,
                colsample_bytree=0.8,
                min_child_weight = 5,
                seed=42)

['transactionDateTime', 'merchantName', 'posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'currentExpDate', 'accountOpenDate', 'dateOfLastAddressChange', 'transactionType']
['accountNumber', 'customerId', 'creditLimit', 'availableMoney', 'transactionAmount', 'cardCVV', 'enteredCVV', 'cardLast4Digits', 'currentBalance', 'cardPresent', 'isFraud']


In [31]:
pd00 = pd.read_csv(WORK_PATH + "/data/processed_data.csv")
print(pd00.shape)

(786363, 26)


In [43]:
pd00.groupby('merchantCategoryCode').agg({'isFraud':np.mean}).reset_index()

,merchantCategoryCode,isFraud
0,airline,0.034648
1,auto,0.012609
2,cable/phone,0.000000
3,entertainment,0.011998
4,fastfood,0.009577
5,food,0.013432
6,food_delivery,0.000000
7,fuel,0.000000
8,furniture,0.013859
9,gym,0.000000


In [49]:
from collections import Counter
val = Counter(pd00['merchantCategoryCode'].tolist()).most_common(1)[0][0]
val

'online_retail'

In [54]:
freq_items = pd00['merchantCategoryCode'].value_counts().index.tolist()[:15]
def relabel(x):
    if x in freq_items:
        return x
    else:
        return 'other'
pd00['merchantCategoryCode'] = pd00['merchantCategoryCode'].apply(relabel )
pd00 = pd.get_dummies(pd00,  columns =  ['merchantCategoryCode'] )

In [35]:
pd00 = pd.get_dummies(pd00,  columns =  ['posEntryMode'] )
pd00.columns.tolist()

['accountNumber',
 'customerId',
 'creditLimit',
 'availableMoney',
 'transactionDateTime',
 'transactionAmount',
 'merchantName',
 'posConditionCode',
 'merchantCategoryCode',
 'currentExpDate',
 'accountOpenDate',
 'dateOfLastAddressChange',
 'cardCVV',
 'enteredCVV',
 'cardLast4Digits',
 'transactionType',
 'currentBalance',
 'cardPresent',
 'isFraud',
 'date',
 'tvt_code_0',
 'tvt_code_1',
 'tvt_code_2',
 'tvt_code_3',
 'tvt_code_4',
 'posEntryMode_2.0',
 'posEntryMode_5.0',
 'posEntryMode_9.0',
 'posEntryMode_80.0',
 'posEntryMode_90.0']

In [55]:
new_feats =  [col for col in pd00.columns.tolist() if "posEntryMode" in col or "merchantCategoryCode" in col ] 
feats = ['creditLimit', 'availableMoney', 'transactionAmount','cardPresent'] 
all_feats = new_feats + feats
x_train = pd00[pd00['tvt_code_0']=='train_0'][all_feats]
y_train = pd00[pd00['tvt_code_0']=='train_0']['isFraud']

x_val = pd00[pd00['tvt_code_0']=='val_0'][all_feats]
y_val = pd00[pd00['tvt_code_0']=='val_0']['isFraud']

x_test = pd00[pd00['tvt_code_0']=='test'][all_feats]
y_test = pd00[pd00['tvt_code_0']=='test']['isFraud']



In [58]:
clf.fit(x_train, y_train, early_stopping_rounds=50, eval_metric="auc", verbose=200,
        eval_set=[(x_val, y_val)])

[0]	validation_0-auc:0.71063
Will train until validation_0-auc hasn't improved in 50 rounds.
[200]	validation_0-auc:0.77948
Stopping. Best iteration:
[315]	validation_0-auc:0.78055



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.035, max_delta_step=0, max_depth=10,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=42, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [59]:
y_pred = clf.predict_proba(x_test)[:,1]
roc_auc_score(y_test, y_pred)

0.7788256646924123